In [80]:
#from __future__ import print_function
from pandas import DataFrame
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [81]:
class DataProvider:
    #Args: path to excel file (standard structure)
    #Provide specific DataFrames (students points per ex,students points per ex part, students marks, teacher list)
    def __init__(self,data_path):
        #Import raw data (max points and students points)
        #Exercises parts
        raw_es_parts=df_data= pd.read_excel(data_path,header=1,usecols="A:AA",engine='openpyxl').dropna()
        #Exercises
        raw_es=pd.read_excel(data_path,header=1,usecols="A,AB:AG",engine='openpyxl').dropna()
        #Exam marks vs year marks
        raw_marks=pd.read_excel(data_path,header=1,usecols="A,AJ:AK",engine='openpyxl').dropna()
        
        #Student's points per part with column 'Docente' 
        self.df_es_parts = raw_es_parts.iloc[1:,:]
        #Student's points per ex with column 'Docente' 
        self.df_es = raw_es.iloc[1:,:]
        
        #Teacher list
        self.teachers=self.df_es.Docente.unique().tolist()
        
        #Student's marks  with column 'Docente' 
        self.df_marks = raw_marks.iloc[1:,:]
        #Student's marks  without column 'Docente' 
        self.df_marks_all=self.df_marks.iloc[:,1:]
        #Dictionary: all, <teacher>. With column Docente for comparison (color='Docente')
        self.dict_marks_teachers = {}
        for teacher in self.teachers:
            self.dict_marks_teachers[teacher]=self.df_marks[self.df_marks['Docente']==teacher]
        self.dict_marks_teachers['all']=self.df_marks.replace({'Docente': r'[\w\s]+'}, {'Docente': 'all'}, regex=True)
        
        
        #Dataframe Maximum points per part
        self.df_pts_parts = raw_es_parts.iloc[0:1,1:]
        #List Maximum points per part
        self.pts_parts_list =  raw_es_parts.iloc[0,1:]
        #DataFrame Maximum points per es
        self.df_pts_es = raw_es.iloc[0:1,1:]
        self.pts_es_list = raw_es.iloc[0,1:]

        
        #Dictionary: all, <teacher>. With column Docente for comparison (color='Docente')
        self.df_es_teachers = {}
        for teacher in self.teachers:
            self.df_es_teachers[teacher]=self.df_es[self.df_es['Docente']==teacher]
        self.df_es_teachers['all']=self.df_es.replace({'Docente': r'[\w\s]+'}, {'Docente': 'all'}, regex=True)
    

        #Dictionary: all, <teacher>. With column Docente for comparison (color='Docente')
        self.df_parts_teachers = {}
        for teacher in self.teachers:
            self.df_parts_teachers[teacher]=self.df_es_parts[self.df_es_parts['Docente']==teacher]
        self.df_parts_teachers['all']=self.df_es_parts.replace({'Docente': r'[\w\s]+'}, {'Docente': 'all'}, regex=True)

        #DataFrame Es points adn parts (without Docenti column)
        self.df_es_all= raw_es.iloc[1:,1:]
        self.df_parts_all= raw_es_parts.iloc[1:,1:]
        
        self.dict_ex_all={}
        for ex in range(1,len(self.pts_es_list)+1):
            self.dict_ex_all[ex]=self.df_parts_all.filter(regex=("Es"+str(ex)+"-."))
            
        self.dict_ex_pts={}
        for ex in range(1,len(self.pts_es_list)+1):
            self.dict_ex_pts[ex]=self.df_pts_parts.filter(regex=("Es"+str(ex)+"-."))
            
        
            
    def get_teacher_ex(self,ex,teacher='all'):
        return self.df_parts_teachers[teacher].filter(regex=("Docente|Es"+str(ex)+"-."))
    
    def get_pts_frequency_ex_teacher(self, ex, teacher='all'):
        counts=[]
        for c in dp.get_teacher_ex(ex,teacher).iloc[:,1:].columns: 
            out=widgets.Output()
            with out:
                display(dp.get_teacher_ex(ex,teacher)[c].value_counts().to_frame())
            counts.append(out)
        box=widgets.HBox(counts)
        display(box)

In [82]:
dp=DataProvider('./data.xlsx')
#dp.get_pts_frequency_ex_teacher(4,'Fulvio Frapolli')
#for c in dp.get_teacher_ex(6,'Fulvio Frapolli').columns: 
#    display(dp.get_teacher_ex(6,'Fulvio Frapolli')[c].value_counts().to_frame())

In [83]:
def display_all_parts(DataProvider,teacher):
    dp=DataProvider
    df_all=dp.df_parts_all
    if teacher =='all':
        fig=px.box(df_all)

        
    else:
        df_compare=pd.concat([dp.df_parts_teachers[teacher],dp.df_parts_teachers['all']])
        fig=px.box(df_compare,color='Docente')
        
    
    
    fig.add_trace(
        go.Scatter(
            x=df_all.columns,
            y=dp.pts_parts_list,
            mode="markers",
            name="max Pts",
            marker=dict(size=12,
            symbol="star"))
        )
    #part_boxplot_widget.clear_output()  
    
    #with part_boxplot_widget:
    fig.update_layout( 
        xaxis_title="Esercizi",
        yaxis_title="Punti",
        #title="Esercizi"
    ) #Definisce titolo asse x , y e principale
    fig.update_xaxes(
        tickangle = -45,    
    )
    fig.show()
 

In [84]:
def display_all_ex(DataProvider,teacher):
    dp=DataProvider
    df_all=dp.df_es_all
    if teacher =='all':
        fig=px.box(df_all)

        
    else:
        df_compare=pd.concat([dp.df_es_teachers[teacher],dp.df_es_teachers['all']])
        fig=px.box(df_compare,color='Docente')
        
    
    
    fig.add_trace(
        go.Scatter(
            x=df_all.columns,
            y=dp.pts_es_list,
            mode="markers",
            name="max Pts",
            marker=dict(size=12,
            symbol="star"))
        )
    #es_boxplot_widget.clear_output()  
    
    #with es_boxplot_widget:
    fig.update_layout( 
        xaxis_title="Esercizi",
        yaxis_title="Punti",
        #title="Esercizi"
    ) #Definisce titolo asse x , y e principale
    fig.update_xaxes(
        tickangle = -45,    
    )
    fig.show()
  


In [85]:
def display_one_ex_parts(DataProvider,teacher,ex=1,count=False):
    dp=DataProvider
    df=dp.dict_ex_all[ex]
    if teacher =='all':
        
        fig=px.box(df)

        
    else:
        #TODO
        df_compare=pd.concat([dp.get_teacher_ex(ex,teacher),dp.get_teacher_ex(ex,'all')])
        fig=px.box(df_compare,color='Docente')

    
    
    fig.add_trace(
        go.Scatter(
            x=df.columns,
            y=dp.dict_ex_pts[ex].iloc[0,:],
            mode="markers",
            name="max Pts",
            marker=dict(size=12,
            symbol="star"))
        )
    #one_ex_parts_widget.clear_output()  
    
    #with one_ex_parts_widget:
    fig.update_layout( 
        xaxis_title="Esercizi",
        yaxis_title="Punti",
        #title="Esercizi"
    ) #Definisce titolo asse x , y e principale
    fig.update_xaxes(
        tickangle = -45,    
    )
    fig.show()
    
    if count:
        dp.get_pts_frequency_ex_teacher(ex,teacher) 

In [86]:
def display_all_marks(DataProvider,teacher):
    dp=DataProvider
    df=dp.df_marks_all
    if teacher =='all':
        
        fig=px.box(df)
    else:
        df_compare=pd.concat([dp.dict_marks_teachers[teacher],dp.dict_marks_teachers['all']])
        fig=px.box(df_compare,color='Docente')
    
    
    fig.update_layout( 
        xaxis_title="Esercizi",
        yaxis_title="Nota",
        #title="Esercizi"
    ) #Definisce titolo asse x , y e principale
    fig.update_xaxes(
       # tickangle = -45,    
    )
    fig.update_yaxes(range=[1, 6])
    fig.show()

In [87]:
def display_compare(DataProvider,teacher1,teacher2):
    dp= DataProvider
    df_compare=pd.concat([dp.df_es_teachers[teacher1],dp.df_es_teachers[teacher2]])
    
    fig=px.box(df_compare,color='Docente')
    fig.update_layout( 
        xaxis_title="Esercizi",
        yaxis_title="Nota",
        #title="Esercizi"
    ) #Definisce titolo asse x , y e principale
    fig.update_xaxes(
        tickangle = -45,    
    )
    fig.show()

In [88]:


children = []
out_all_parts=widgets.Output()
out_sub_es=widgets.Output()
out_es=widgets.Output()
out_marks=widgets.Output()
out_compare=widgets.Output()
children.append(out_all_parts)
children.append(out_sub_es)
children.append(out_es)
children.append(out_marks)
children.append(out_compare)

tab = widgets.Tab()
tab.children = children
tab.set_title(0,'All Parts')
tab.set_title(1,'1 Ex - All parts')
tab.set_title(2,'All Ex')
tab.set_title(3,'Note')
tab.set_title(4,'Comparativo 1-1')


with out_sub_es:
    #one_ex_parts_widget =  widgets.Output()
    widgets.interact(
        display_one_ex_parts, 
        DataProvider=fixed(dp),
        ex=widgets.IntSlider(min=1, max=6, step=1, value=1,description="Esercizio"), 
        teacher=widgets.Dropdown(options=['all']+dp.teachers, description='Docente:',disabled=False,),
        count=widgets.Checkbox(value=False, description='Frequenza', disabled=False)
        )
    #display(one_ex_parts_widget)
    
with out_all_parts:
    #part_boxplot_widget = widgets.Output()
    #Interazione con la funzione che sisegna il BoxPlot , es=esercizio da viualizzare, docente|all
    widgets.interact(
        display_all_parts, 
        DataProvider=fixed(dp),
        #es=widgets.IntSlider(min=1, max=6, step=1, value=1,description="Esercizio"), 
        teacher=widgets.Dropdown(options=['all']+dp.teachers, description='Docente:',disabled=False,),
        )

    #display(part_boxplot_widget)
with out_es:
    #es_boxplot_widget=widgets.Output()
    widgets.interact(
        display_all_ex,
        DataProvider=fixed(dp),
        teacher=widgets.Dropdown(options=['all']+dp.teachers, description='Docente:',disabled=False,)
        )

    #display(es_boxplot_widget)
with out_marks:
    widgets.interact(
        display_all_marks,
        DataProvider=fixed(dp),
        teacher=widgets.Dropdown(options=['all']+dp.teachers, description='Docente:',disabled=False,)
        )
    


with out_compare:
    widgets.interact(
        display_compare,
        DataProvider=fixed(dp),
        teacher1=widgets.Dropdown(options=['all']+dp.teachers, description='Docente1:',disabled=False,),
        teacher2=widgets.Dropdown(options=['all']+dp.teachers, description='Docente2:',disabled=False,)
        )
    

    
    
tab